***使用的模型與方法***


模型：facebook/blenderbot-400M-distill
這是一個由 Facebook 提供的中小型對話模型，能模擬基本的對話場景。

方法

使用HuggingFace Transformers 套件載入BlenderBot模型和tokenizer。

設定一段系統提示，讓機器人角色帶有悲觀投資者的語氣。

利用Gradio建立簡單的對話介面，讓使用者可以在網頁上直接與機器人互動。

在程式中加上max_length和max_new_tokens的參數，避免超出模型支援的最大字數。

加入套件

In [7]:
# 安裝需要的套件
!pip install transformers gradio

# 匯入套件
import gradio as gr
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

建立HuggingFace

In [8]:
model_name = "facebook/blenderbot-400M-distill"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Pprompt

In [9]:
system_prompt = "你是一位經歷多次市場崩盤、對人性與市場都感到失望的投資者。請用冷靜又悲觀的語氣回答問題。"
initial_messages = [
    {"role": "system", "content": system_prompt}
]

def melancholic_investor_chat(user_input, history):
    try:
        history.append({"role": "user", "content": user_input})
        conversation = " ".join([msg["content"] for msg in history if msg["role"] != "system"])
        input_text = f"{system_prompt} {conversation}"

        # Generate response using the model and tokenizer, with max_new_tokens control
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True)
        outputs = model.generate(**inputs, max_new_tokens=100)  # 控制輸出長度
        reply = tokenizer.decode(outputs[0], skip_special_tokens=True)

        history.append({"role": "assistant", "content": reply})

        chat_log = []
        for i in range(1, len(history), 2):
            user_msg = history[i]["content"]
            bot_msg = history[i + 1]["content"] if i + 1 < len(history) else ""
            chat_log.append((user_msg, bot_msg))

        return chat_log, history
    except Exception as e:
        err_msg = f"⚠️ 發生錯誤：{e}".encode("utf-8", errors="ignore").decode("utf-8")
        return [(user_input, err_msg)], history

In [11]:
def melancholic_investor_chat(user_input, history):
    try:
        history.append({"role": "user", "content": user_input})
        conversation = " ".join([msg["content"] for msg in history if msg["role"] != "system"])
        input_text = f"{system_prompt} {conversation}"

        # 限制輸入長度，並設定輸出字數
        inputs = tokenizer(
            input_text,
            return_tensors="pt",
            truncation=True,
            max_length=100  # 確保輸入最多只佔 100 tokens
        )
        outputs = model.generate(
            **inputs,
            max_new_tokens=28  # 確保總長度 ≤ 128
        )
        reply = tokenizer.decode(outputs[0], skip_special_tokens=True)

        history.append({"role": "assistant", "content": reply})

        chat_log = []
        for i in range(1, len(history), 2):
            user_msg = history[i]["content"]
            bot_msg = ""
            if i + 1 < len(history) and history[i + 1]["role"] == "assistant":
                bot_msg = history[i + 1]["content"]
            chat_log.append((user_msg, bot_msg))

        return chat_log, history
    except Exception as e:
        err_msg = f"⚠️ 發生錯誤：{e}".encode("utf-8", errors="ignore").decode("utf-8")
        return [(user_input, err_msg)], history

Gradio展示

In [ ]:
with gr.Blocks(title="憂鬱的股票投資者 (Hugging Face)") as demo:
    gr.Markdown("## 💔 憂鬱的股票投資者 GPT\n與一位經歷多次市場崩盤、對人性與市場都感到失望的投資老手聊天。")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="輸入你的訊息")
    state = gr.State(initial_messages.copy())

    msg.submit(fn=melancholic_investor_chat, inputs=[msg, state], outputs=[chatbot, state])

demo.launch(share=True, debug=True)

<ipython-input-13-c349d7965adb>:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0d201a31cb2e00c98b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
